## preprocesing tekstu

In [10]:

import numpy as np
from sortedcontainers import SortedSet
alphabet = SortedSet()
filename = "lines.txt"
lines = None

def preprocess(text: str) -> str:
    text = text.lower()
    marks = ["!", "\"", "#", "$", "%", "&", "'", "(", ")", "*", ",", ".", "/", ":", ";", "<", "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "{", "|", "}", "~"]
    for mark in marks:
        text = text.translate({ord(mark):" "})
    # Your code here:
    # Convert the text to lowercase.
    # Remove all punctuation marks;
    return text

def preprocess_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    lines = list(map(preprocess, lines))
    for line in lines:
        alphabet.update(line.split())
    return np.array(lines)
lines = preprocess_file(filename)

def text_to_vec(docs: list[str]) -> list[list[int]]:
    # Your code here:
    # Convert documents to numerical vectors.
    # Preprocess documents with the preprocess() function.
    # Represent documents as vectors of word frequencies, 
    # you will need to extract a vocabulary from all the documents.
    sentences = []

    for doc in docs:
        sentences.append([])
        sentences[-1] = preprocess(doc).split()
        
    freq_vecs = []    
    n = len(alphabet)
    
    for sentence in sentences:
        freq_vecs.append([0] * n)
        for word in sentence:
            freq_vecs[-1][alphabet.index(word)] += 1

    return freq_vecs

## metryk w przestrzeni napisów

In [43]:
#Kod przekopiowany z ćwiczeń
def cosine_similarity(vec_a, vec_b) -> float:
    cosine_similarity = 0
    
    cosine_similarity = vec_a.dot(vec_b)/(np.linalg.norm(vec_a)*np.linalg.norm(vec_b))
    return cosine_similarity

def dice(text_a: str, text_b: str) -> float:
    dice = 0
    set_A = set(preprocess(text_a).split())
    set_B = set(preprocess(text_b).split())
    dice = 2*len(set_A.intersection(set_B)) / (len(set_A) + len(set_B))
    return dice

def euclidean_distance(vec_a, vec_b) -> float:
    dist = 0
    dist = np.linalg.norm(vec_a - vec_b)
    return dist

 ## indeks Daviesa-Bouldina

In [3]:
def dbi(data, cluster_classification, cluster_center, distance_func):
    #ilość klastrów
    k = len(cluster_center)
    
    #średnia odległość punktów w klastrze od ośrodka ciężkości odpowiadającego klastra
    sigma = np.zeros(k)
    for i in range(k):
        sigma[i] = np.mean([distance_func(data[j], cluster_center[i]) for j in np.where(cluster_classification == i)[0]])

    # obliczanie odległości od siebie klastrów
    R = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            if i != j:
                R[i][j] = (sigma[i] + sigma[j]) / distance_func(cluster_center[i], cluster_center[j])

    # obliczanie indewksu dbi
    dbi = 0
    for i in range(k):
        dbi += np.max([R[i][j] for j in range(k) if j != i])
    
    return dbi / k

## tworzenie stoplisty

In [20]:
from sortedcontainers import SortedDict

def count_the_words(lines):
    word_counter = SortedDict()
    for line in lines:
        for word in line.split():
            if word in word_counter.keys():
                word_counter[word] += 1
            else:
                word_counter[word] = 0
    return word_counter

t = sorted(list(count_the_words(lines).items()), key=lambda x:-x[1])

In [21]:
## po analizie powyrzszego wyniku zdecydowałem się na tą stopliste:
stoplist = {"the", "and", "on", "to", "rm", "com"}
## oraz wszystkie pojedyńcze znaki
for line in lines:
    for word in line.split():
        if len(word) <= 1:
            stoplist.add(word)
stoplist


{'+',
 '-',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'and',
 'b',
 'c',
 'com',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'on',
 'p',
 'q',
 'r',
 'rm',
 's',
 't',
 'the',
 'to',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [22]:
alphabet_stoplist = SortedSet()
for line in lines:
    for word in line.split():
        if word not in stoplist:
            alphabet_stoplist.add(word)

def text_to_vec_stoplist(docs: list[str]) -> list[list[int]]:
    # Your code here:
    # Convert documents to numerical vectors.
    # Preprocess documents with the preprocess() function.
    # Represent documents as vectors of word frequencies, 
    # you will need to extract a vocabulary from all the documents.
    sentences = []

    for doc in docs:
        sentences.append([])
        sentences[-1] = preprocess(doc).split()
        
    freq_vecs = []    
    n = len(alphabet_stoplist)
    
    for sentence in sentences:
        freq_vecs.append([0] * n)
        for word in sentence:
            if word not in stoplist:
                freq_vecs[-1][alphabet_stoplist.index(word)] += 1

    return freq_vecs

# klastrowanie:

In [23]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

# zakładam że na klaster przypadnie 100 adresów
cluster_number = len(lines) // 100

lines_vector = text_to_vec(lines)
lines_vector_stoplist = text_to_vec_stoplist(lines)

## klastry bez preprocesingu

In [32]:
kmeans1 = KMeans(n_clusters=cluster_number, random_state=0)

kmeans1.fit(lines_vector)

centroids = kmeans1.cluster_centers_

labels1 = kmeans1.predict(lines_vector)

/home/ciastek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [33]:
clusters = []
clusters_vec = []

for i in range(cluster_number):
    clusters.append(np.where(labels1==i, lines, None))


for cluster in clusters:
    print("#######################################################")
    for sencence in cluster:
        if sencence is not None:
            print(sencence, end="")
    print()

#######################################################
h m hennes mauritz gbc ab  ul magazynova 3 pl-62 023 gadki  poznan  e-mail  dlimpex-pl hm com phone +48  0 61 898 94 24
h m hennes mauritz gbc ab  ul magazynova 3 pl-62 023 gadki  poznan  e-mail  dlimpex-pl hm com phone +48  0 61 898 94 24
h m hennes mauritz gbc ab  ul magazynova 3 pl-62 023 gadki  poznan  e-mail  dlimpex-pl hm com phone 48 0 61 898 94 24
h m hennes mauritz gbc ab ul  magazynova 3 pl-62 023 gadki  poznan  e-mail  dlimpex-pl hm com phone +48  0 61 898 94 24
h m hennes mauritz gbc ab ul  magazynova 3 pl-62 023 gadki  poznan  e-mail  dlimpex-pl hm com phone +48  0 61 898 94 24
h m hennes mauritz logistic gbcab ul  magazynova 3 pl-62 023 gadki  poznan      e-mail  dlimpex-pl hm com phone  +48  0 61 898 94 24
h m hennes mauritz logistic spz o o  magazynova 3 pl-62 023 gadki poznan  e-mail  dlimpex-pl hm comphone 48  0 61 898 94 24
h m hennes mauritz logistic spz o o  magazynova 3 pl-62 023 gadki  poznan  poland
h m hen

In [46]:
from sklearn.metrics import davies_bouldin_score

dbi1 = davies_bouldin_score(lines_vector, labels1)

print('DBI:', dbi1)
print("my_DBI", dbi(lines_vector, labels1, centroids, euclidean_distance))


DBI: 2.7972486384613076
my_DBI 2.7972486384613


## klastry z stoplistą

In [39]:
kmeans2 = KMeans(n_clusters=cluster_number, random_state=0)

kmeans2.fit(lines_vector_stoplist)

centroids_stoplist = kmeans2.cluster_centers_

labels2 = kmeans2.predict(lines_vector_stoplist)

/home/ciastek/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [40]:
clusters_stoplist = []

for i in range(cluster_number):
    clusters_stoplist.append(np.where(labels2==i, lines, None))


for cluster in clusters_stoplist:
    print("#######################################################")
    for sencence in cluster:
        if sencence is not None:
            print(sencence, end="")
    print()

#######################################################
to order of unilever supply chain company ag   spitalstrasse 5 postfach  schaffhausen 8201  ch tel +41526315000  fax +41526315500
unilever supply chain co  ag account payable dept  spital strasse 5  postfach schaffhausen 8201 ch +41526315000 +41526315500
unilever supply chain co ag accountpayable dept spital strasse 5  postfach schaffhausen 8201 ch +41526315000 +41526315500
unilever supply chain co ag accountpayable dept spital strasse 5  schaffhausen 8201 ch +41526315000 +41526315500
unilever supply chain company ag  spitalstrasse 5 schaffhausen 8201 switzerland  tel  +41526315000  fax +41526315500
unilever supply chain company ag  spitalstrasse 5 schaffhausen 8201 switzerland  tel  +41526315000 fax  +41526315500
unilever supply chain company ag  accounts payable  dept  spitalstrasse 5 schaffhausen 8201 ch 41526315000  41526315500
unilever supply chain company ag  spitalstrasse 5 schaffhausen 8201 switzerland telp   +41526315000 

In [41]:
from sklearn.metrics import davies_bouldin_score

dbi2 = davies_bouldin_score(lines_vector, labels2)

print('DBI:', dbi2)

DBI: 2.831372752359995
